# Reasoning behind data cleaning

## Plain English Summary

This notebook explains the decisions made when cleaning SSNAP extract v2.

## Aims

* Explain decisions related to variables that were included in the clean dataset
* Explain why some variables from the SSNAP data were not included in the clean dataset

## Observations

tbc

## Set up

In [1]:
# Import packages and functions
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from dataclasses import dataclass

# Linting
%load_ext pycodestyle_magic
%pycodestyle_on

# Set the maximum number of columns and rows to 150
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)


# Set paths and filenames
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and files.'''

    data_path = './../output/'
    data_filename = 'reformatted_data.csv'
    raw_path = './../data'
    raw_filename = 'SAMueL ssnap extract v2.csv'
    notebook = '01'


paths = Paths()

In [2]:
# Load data
data = pd.read_csv(os.path.join(paths.data_path, paths.data_filename))
raw_data = pd.read_csv(os.path.join(paths.raw_path, paths.raw_filename),
                       low_memory=False)

In [3]:
def compare_raw_clean(raw_data, clean_data):
    '''
    Merges the raw and clean data using patient ID
    Then presents counts for the remaining columns when ID is dropped
    '''
    compare = pd.merge(left=clean_data, left_on='id',
                       right=raw_data, right_on='PatientId')
    differences = (compare
                   .drop(columns=['id', 'PatientId'], axis=1)
                   .value_counts(dropna=False)
                   .reset_index(name='count'))
    return (differences.sort_values(differences.columns[0]))


def mark_missing(df):
    '''
    Identifies NaN then marks each value as missing data or having data
    '''
    df_missing = (pd.DataFrame(df)
                  .isnull()
                  .apply(lambda x: x.map({True: 'Missing data',
                                          False: 'Have data'})))
    return (df_missing)

## Included variables

### Overview of dataset

In [4]:
# Show shape of the data
data.shape

(358993, 70)

In [5]:
# List all columns and show their data types and proportion of completed data
data_type_complete = pd.DataFrame(
    {'Data type': data.dtypes,
     'Proportion complete': data.count() / data.shape[0]})

# Show all columns
data_type_complete

,Data type,Proportion complete
id,int64,1.000000
stroke_team,object,1.000000
age,float64,1.000000
male,int64,1.000000
infarction,float64,1.000000
onset_to_arrival_time,int64,1.000000
onset_known,int64,1.000000
precise_onset_known,int64,1.000000
onset_during_sleep,int64,1.000000
arrive_by_ambulance,int64,1.000000


### ID

The patient ID from the SSNAP dataset is included, as this provides a consistent identifier between the raw data and different/cleaned versions of the data.

### Age

The SSNAP data provides age and arrival time as categories. These were converted to numerical variables based on the middle value in each category. We do not have data on what the average age or arrival time was for each category, so this is an assumption. There is a chance that, for example, for ages 90 or over, the average age is actually lower than 92.5. However, in lieu of that information, this is considered a reasonable guess.

In [6]:
# Convert raw age to single column
raw_ages = pd.DataFrame(raw_data['PatientId'])
raw_ages['raw_age'] = raw_data[[
    col for col in raw_data if col.startswith('Age')]].idxmax(1)

# Merge with clean and present difference
compare_raw_clean(raw_ages, data[['id', 'age']])

,age,raw_age,count
10,37.5,AgeUnder40,6110
11,42.5,Age40to44,5072
9,47.5,Age45to49,9314
8,52.5,Age50to54,15839
7,57.5,Age55to59,21285
6,62.5,Age60to64,26881
5,67.5,Age65to69,33609
3,72.5,Age70to74,45959
1,77.5,Age75to79,52542
0,82.5,Age80to84,56912


In [7]:
# Sequence of numbers showing the mid-points
np.arange(35, 95, 2.5)

array([35. , 37.5, 40. , 42.5, 45. , 47.5, 50. , 52.5, 55. , 57.5, 60. ,
       62.5, 65. , 67.5, 70. , 72.5, 75. , 77.5, 80. , 82.5, 85. , 87.5,
       90. , 92.5])

### Gender

No comment - as in SSNAP dataset.

### Stroke type

It is vital that we understand stroke type, so patients were missing stroke type (missing due to having not received a scan) were completely removed from the clean dataset.

In [8]:
# Show relationship between missing stoke type and missing arrival to scan time
(mark_missing(raw_data[['S2StrokeType', 'ArrivaltoBrainImagingMinutes']])
 .value_counts()
 .reset_index(name='count'))

,S2StrokeType,ArrivaltoBrainImagingMinutes,count
0,Have data,Have data,358993
1,Missing data,Missing data,1388


### Ambulance times

In cases where any ambulance times are missing or inappropriate, all ambulance times are set as missing.

In [9]:
# Extract ambulance times
raw_amb_times = raw_data[[
    'CallConnectedtoArrivalMinutes',
    'ArrivalPatientLocationtoArrivalMinutes',
    'DeparturePatientLocationtoArrivalMinutes',
    'WheelsStoptoArrivalMinutes']]
clean_amb_times = data[[
    'call_to_ambulance_arrival_time',
    'ambulance_on_scene_time',
    'ambulance_travel_to_hospital_time',
    'ambulance_wait_time_at_hospital']]

**Missing times**

The clean dataset either requires all times to be complete or all to be missing.

In [10]:
# Presence of missing ambulance times in clean data
(clean_amb_times
 .isnull()
 .apply(lambda x: x.map({True: 'Missing time', False: 'Has time'}))
 .value_counts()
 .reset_index(name='count'))

,call_to_ambulance_arrival_time,ambulance_on_scene_time,ambulance_travel_to_hospital_time,ambulance_wait_time_at_hospital,count
0,Missing time,Missing time,Missing time,Missing time,272906
1,Has time,Has time,Has time,Has time,86087


In [11]:
# Missing ambulance times in raw data
(raw_amb_times
 .isnull()
 .apply(lambda x: x.map({True: 'Missing time', False: 'Has time'}))
 .value_counts()
 .reset_index(name='count'))

,CallConnectedtoArrivalMinutes,ArrivalPatientLocationtoArrivalMinutes,DeparturePatientLocationtoArrivalMinutes,WheelsStoptoArrivalMinutes,count
0,Missing time,Missing time,Missing time,Missing time,251882
1,Has time,Has time,Has time,Has time,92958
2,Has time,Has time,Has time,Missing time,15421
3,Has time,Has time,Missing time,Missing time,87
4,Has time,Missing time,Missing time,Has time,8
5,Has time,Missing time,Missing time,Missing time,7
6,Missing time,Has time,Has time,Has time,5
7,Missing time,Has time,Missing time,Missing time,5
8,Has time,Missing time,Has time,Has time,3
9,Missing time,Has time,Has time,Missing time,3


**Negative times**

We get negative times when:
* A time is greater than the subsequent time in the sequence (e.g. call_to_ambulance_arrival is ArrivalPatient - CallConnected, but will be negative if CallConnected is larger than ArrivalPatient)
* WheelsStoptoArrivalMinutes is positive (as calculated by 0 - WheelsStop)

The clean dataset contains no negative times.

In [12]:
(raw_data['CallConnectedtoArrivalMinutes'] >
 raw_data['ArrivalPatientLocationtoArrivalMinutes']).value_counts()

False    360309
True         72
dtype: int64

In [13]:
(raw_data['ArrivalPatientLocationtoArrivalMinutes'] >
 raw_data['DeparturePatientLocationtoArrivalMinutes']).value_counts()

False    360229
True        152
dtype: int64

In [14]:
(raw_data['DeparturePatientLocationtoArrivalMinutes'] >
 raw_data['WheelsStoptoArrivalMinutes']).value_counts()

False    360011
True        370
dtype: int64

In [15]:
(raw_data['WheelsStoptoArrivalMinutes'] > 0).value_counts()

False    354343
True       6038
Name: WheelsStoptoArrivalMinutes, dtype: int64

In [16]:
(clean_amb_times < 0).value_counts().reset_index(name='count')

,call_to_ambulance_arrival_time,ambulance_on_scene_time,ambulance_travel_to_hospital_time,ambulance_wait_time_at_hospital,count
0,False,False,False,False,358993


**Times equal to zero**

It is plausible that the wait time at the hospital might be 0 minutes. However, a time of 0 minutes is not plausible for:
* Call received to ambulance arrival
* Length of time that ambulance is on scene
* Travel time to hospital

Hence, in those cases, all times are set as NaN.

In [17]:
(clean_amb_times == 0).value_counts().reset_index(name='count')

,call_to_ambulance_arrival_time,ambulance_on_scene_time,ambulance_travel_to_hospital_time,ambulance_wait_time_at_hospital,count
0,False,False,False,False,347866
1,False,False,False,True,11127


**Unreasonably large times**

In [18]:
# Unreasonably large ambulance times

### Thrombectomy

If ArrivaltoArterialPunctureMinutes is NaN, we conclude they did not have a thrombectomy. Is there a possibility that this could be missing for anyone who did have a thrombectomy? No, the SSNAP team suggested that this is how we identify people who have had a thrombectomy, and that everyone who had one should have a time.

In [19]:
# Show where raw data had or was missing data, then compare with clean data
raw_thrombectomy = mark_missing(raw_data['ArrivaltoArterialPunctureMinutes'])
raw_thrombectomy['PatientId'] = raw_data['PatientId']
compare_raw_clean(raw_thrombectomy,
                  data[['id', 'thrombectomy']])

,thrombectomy,ArrivaltoArterialPunctureMinutes,count
0,0,Missing data,354650
1,1,Have data,4343


### Reasons for no thrombolysis

No comment - as in SSNAP dataset.

### Arrival time

The same logic to convert age from a categorical to numeric variable was used to convert arrival times - setting the value to the middle of the bracket. Hence, same caveat with this assumption applies - that e.g. the average arrival time for 12am to 3am is not actually 1.30am.

In [20]:
# Extract relevant columns and merge
compare_time = pd.merge(
    left=data[['id', 'arrival_time_3_hour_period']], left_on='id',
    right=raw_data[['PatientId', 'FirstArrivalTime']], right_on='PatientId')

# Table illustrating change from categorical to numerical variable
(compare_time
 .drop(['PatientId', 'id'], axis=1)
 .value_counts(dropna=False)
 .reset_index(name='count')
 .sort_values(by='arrival_time_3_hour_period'))

,arrival_time_3_hour_period,FirstArrivalTime,count
6,1.5,0000to3000,15628
7,4.5,0300to0600,11481
5,7.5,0600to0900,24980
0,10.5,0900to1200,81705
1,13.5,1200to1500,78339
2,16.5,1500to1800,64213
3,19.5,1800to2100,50236
4,22.5,2100to2400,32411


In [21]:
# Sequence of numbers showing the mid-points
np.arange(0, 24, 1.5)

array([ 0. ,  1.5,  3. ,  4.5,  6. ,  7.5,  9. , 10.5, 12. , 13.5, 15. ,
       16.5, 18. , 19.5, 21. , 22.5])

### Anticoagulants and antiplatelets

Among possible answers to these variables in the raw data were no (N) and no but (NB) - we are not aware what the difference between these are, but see no reason why there would be an issue of setting both as 0 in the clean data.

<mark>Note:</mark> Although named AFAnticoagulent, not contingent on AtrialFibrillation=Yes in data dictionary and the SSNAP form - that is only for antiplatelets. Why is there contingency for that and not anticoagulants in SSNAP?

<mark> Note:</mark> We don't know that NaN definetely means No.

In [22]:
antiplatelet = compare_raw_clean(
        raw_data[['PatientId', 'S2CoMAFAntiplatelet']],
        data[['id', 'atrial_fibrillation', 'afib_antiplatelet']])
antiplatelet.sort_values(by=['atrial_fibrillation', 'afib_antiplatelet'])

,atrial_fibrillation,afib_antiplatelet,S2CoMAFAntiplatelet,count
0,0,0,NaN,293618
1,1,0,N,47231
3,1,0,NB,8153
4,1,0,NaN,1
2,1,1,Y,9990


In [23]:
anticoag = compare_raw_clean(
    raw_data[['PatientId', 'S2CoMAFAnticoagulent']],
    data[['id', 'atrial_fibrillation', 'afib_anticoagulant']])
anticoag.sort_values(by=['atrial_fibrillation', 'afib_anticoagulant'])

,atrial_fibrillation,afib_anticoagulant,S2CoMAFAnticoagulent,count
0,0,0,N,185860
1,0,0,NaN,89650
4,0,0,NB,11326
5,0,1,Y,6782
3,1,0,N,18951
6,1,0,NB,6551
7,1,0,NaN,1
2,1,1,Y,39872


In [24]:
anticoag_vitk = compare_raw_clean(
    raw_data[['PatientId', 'S2CoMAFAnticoagulentVitK']],
    data[['id', 'atrial_fibrillation', 'afib_vit_k_anticoagulant']])
anticoag_vitk.sort_values(by=['atrial_fibrillation',
                              'afib_vit_k_anticoagulant'])

,atrial_fibrillation,afib_vit_k_anticoagulant,S2CoMAFAnticoagulentVitK,count
0,0,0,0.0,200216
1,0,0,NaN,91488
5,0,1,1.0,1914
2,1,0,0.0,37742
3,1,0,NaN,20425
4,1,1,1.0,7208


In [25]:
anticoag_doac = compare_raw_clean(
    raw_data[['PatientId', 'S2CoMAFAnticoagulentDOAC']],
    data[['id', 'atrial_fibrillation', 'afib_doac_anticoagulant']])
anticoag_doac.sort_values(by=['atrial_fibrillation',
                              'afib_doac_anticoagulant'])

,atrial_fibrillation,afib_doac_anticoagulant,S2CoMAFAnticoagulentDOAC,count
0,0,0,0.0,197626
1,0,0,NaN,91488
5,0,1,1.0,4504
2,1,0,0.0,24007
4,1,0,NaN,20425
3,1,1,1.0,20943


In [26]:
anticoag_heparin = compare_raw_clean(
    raw_data[['PatientId', 'S2CoMAFAnticoagulentHeparin']],
    data[['id', 'atrial_fibrillation', 'afib_heparin_anticoagulant']])
anticoag_heparin.sort_values(by=['atrial_fibrillation',
                                 'afib_heparin_anticoagulant'])

,atrial_fibrillation,afib_heparin_anticoagulant,S2CoMAFAnticoagulentHeparin,count
0,0,0,0.0,201897
1,0,0,NaN,91488
4,0,1,1.0,233
2,1,0,0.0,44836
3,1,0,NaN,20425
5,1,1,1.0,114


### New atrial fibrillation diagnosis

Missing data (should be complete for all cases where AtrialFibrillation = N), but still included in cleaned dataset for some descriptive analysis.

In [27]:
((raw_data[['S2CoMAtrialFibrillation', 'S2NewAFDiagnosis']]
  .value_counts(dropna=False)
  .reset_index(name='count')
  .sort_values(by='S2NewAFDiagnosis')))

,S2CoMAtrialFibrillation,S2NewAFDiagnosis,count
0,N,N,185842
3,N,Y,13133
1,N,NaN,95809
2,Y,NaN,65597


### Death

Nearly everyone who is marked to have died also had a Modified Rankin Scale (mRS) score of 6, which indicates death.

<mark>Inconsistencies to consider:</mark>
* There are ten people with mRS=6 but death=0 (could we set death to 1?)
* There is one person with death=1 but mRS=NaN (could we set mRS to 6?)

Or, in both cases, do we have to ignore both results, if we are uncertain which is true?

In [28]:
(data[['death', 'discharge_disability']]
 .value_counts(dropna=False)
 .reset_index(name='count')
 .sort_values(by='discharge_disability'))

,death,discharge_disability,count
4,0,0.0,46395
0,0,1.0,75758
1,0,2.0,65265
2,0,3.0,57517
5,0,4.0,41570
6,0,5.0,20080
3,1,6.0,49839
8,0,6.0,10
7,0,NaN,2557
9,1,NaN,2


## Excluded variables

### INR

**Comparing INR, INR High and INR not known**

* INR High is only marked as true in 31 cases
* In all these cases, they are missing INR result and marked as INR not checked.

Hence, there are serious inconsistencies between the three INR columns.

In [29]:
# Look at proportion with INR high and INR result
inr_high = mark_missing(raw_data['S2INR'])
inr_high['S2INRHigh'] = raw_data['S2INRHigh']
inr_high['S2INRNK'] = raw_data['S2INRNK']
(inr_high[['S2INR', 'S2INRHigh', 'S2INRNK']]
 .value_counts(dropna=False)
 .reset_index(name='count'))

,S2INR,S2INRHigh,S2INRNK,count
0,Missing data,0.0,0.0,238845
1,Missing data,NaN,NaN,112376
2,Have data,0.0,0.0,8506
3,Missing data,0.0,1.0,623
4,Missing data,1.0,0.0,31


**Looking further into INR**

Measurement of INR is contingent on whether patients were on Vitamin K antagonists (inc. Warfarin) before their stroke.

*In relation to Mike's comment that expect INR to be on anticoagulants*

In [30]:
# Find whether missing INR or not, and VitK value
inr = mark_missing(raw_data['S2INR'])
inr['anticoag_vitk'] = raw_data['S2CoMAFAnticoagulentVitK']

# Look at those counts
inr_counts = inr.value_counts(dropna=False).reset_index(name='count')
inr_counts

,S2INR,anticoag_vitk,count
0,Missing data,0.0,238845
1,Missing data,NaN,112376
2,Have data,1.0,8506
3,Missing data,1.0,654


There are equal numbers with best estimate v.s. precise v.s. not known onset, who have INR measurement

*In relation to Mike's comment that wouldn't expect INR measurement when don't know onset*

In [31]:
inr_onset = inr.copy()
inr_onset['S1OnsetTimeType'] = raw_data['S1OnsetTimeType']
(inr_onset[inr_onset['anticoag_vitk'] == 1]
 .value_counts(dropna=False)
 .reset_index(name='count'))

,S2INR,anticoag_vitk,S1OnsetTimeType,count
0,Have data,1.0,BE,2949
1,Have data,1.0,P,2876
2,Have data,1.0,NK,2681
3,Missing data,1.0,BE,258
4,Missing data,1.0,NK,231
5,Missing data,1.0,P,165


For patients who were on Vitamin K antagonists, 92.8% have data and 7.1% are missing data.

In [32]:
# Look at proportion of people on VitK with INR data
vitk = inr_counts[inr_counts['anticoag_vitk'] == 1]
vitk['count'] / vitk['count'].sum()

2    0.928603
3    0.071397
Name: count, dtype: float64

For patients who were on Vitamin K antagonists and have an INR measurement, the number (a) with PIH, and (b) receiving thrombolysis is presented.

*In relation to Mike's comment about relationship between INR measurement, haemorrhage and thrombolysis - and that would expect high INR to not have thrombolysis*

In [33]:
# Add stroke type and thrombolysis
inr_stroke_treatment = inr.copy()
inr_stroke_treatment['thrombolysis'] = raw_data['S2Thrombolysis']
inr_stroke_treatment['stroke'] = raw_data['S2StrokeType']

# Find counts and then restrict to just viewing those with INR data
inr_thrombolysis = (inr_stroke_treatment
                    .value_counts(dropna=False)
                    .reset_index(name='count')
                    .sort_values(by=['S2INR', 'stroke']))
inr_thrombolysis[inr_thrombolysis['S2INR'] == 'Have data']

,S2INR,anticoag_vitk,thrombolysis,stroke,count
6,Have data,1.0,NB,I,6261
13,Have data,1.0,Y,I,353
17,Have data,1.0,N,I,23
7,Have data,1.0,NB,PIH,1840
15,Have data,1.0,NB,NaN,29


### Mobile data to arrival

This is missing for the majority of patients. Everyone who has MobileDatatoArrivalMinutes also have data for CallConnectedtoArrivalMinutes, so this variable is not needed.

*In relation to Mike's comment that presumably everyone with this would also have other data about time to arrival*

In [34]:
(mark_missing(raw_data[['MobileDatatoArrivalMinutes',
                        'CallConnectedtoArrivalMinutes']])
 .value_counts()
 .reset_index(name='count'))

,MobileDatatoArrivalMinutes,CallConnectedtoArrivalMinutes,count
0,Missing data,Missing data,251896
1,Missing data,Have data,92414
2,Have data,Have data,16071


### TIA in last month

This is missing for the majority of patients. Also, it should only be completed if 2.1.5 (had stroke/TIA) is marked as yes, but this is not the case. Hence, not included in clean dataset.

In [35]:
(raw_data[['S2CoMStrokeTIA', 'S2TIAInLastMonth']]
 .value_counts(dropna=False)
 .reset_index(name='count')
 .sort_values(by='S2CoMStrokeTIA'))

,S2CoMStrokeTIA,S2TIAInLastMonth,count
0,N,NaN,268486
5,N,N,342
6,N,NK,42
1,Y,NaN,64115
2,Y,N,22205
3,Y,NK,3845
4,Y,Y,1346


### Stroke unit death

Not included in clean dataset, as we are not currently interested in the location of death.

In [36]:
death_check = pd.DataFrame(raw_data['S7StrokeUnitDeath'])
death_check['death'] = (raw_data['ArrivalToDeathDays'] >= 0) * 1
(death_check
 .value_counts(dropna=False)
 .reset_index(name='count')
 .sort_values(by='death'))

,S7StrokeUnitDeath,death,count
0,NaN,0,310320
4,Y,0,4
1,Y,1,43292
2,NaN,1,4489
3,N,1,2276


### Pre-hospital impression

S1PreHospitalImpression - not certain what it is (might be to do with what paramedic think is the diagnosis - e.g. fast positive) and did not ask for it, not included in cleaned dataset.

## Run unit tests

These perform checks on the cleaned data. Have since learnt these are typically used for functions rather than data cleaning, and there is some redundancy (i.e. tests checking values, when definietely know values as specified when creating the cleaned data). However, since have been created, have kept. Include checks like whether:
* Counts are as expected (i.e. including what intend into 1 and 0)
* Shape of dataframe is consistent (no additional rows or columns added)
* Times are not negative

In [37]:
%run -i './../tests/test_01_reformat_data.py'

...F..
FAIL: test_no_ambulance (__main__.DataTests)
Test that people who do not arrive by ambulance therefore have
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/amy/Documents/samuel_2_data_prep/tests/test_01_reformat_data.py", line 81, in test_no_ambulance
    self.assertEqual(len(amb_neg.index), 0)
AssertionError: 7 != 0

FAIL: test_time_negative (__main__.DataTests) [onset_to_arrival_time]
Test that times are not negative when expected to be positive
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/amy/Documents/samuel_2_data_prep/tests/test_01_reformat_data.py", line 69, in test_time_negative
    self.time_neg(col)
  File "/home/amy/Documents/samuel_2_data_prep/tests/test_01_reformat_data.py", line 35, in time_neg
    self.assertEqual(sum(self.clean[time_column] < 0), 0)
AssertionError: 2 != 0

----------------------------------------------

SystemExit: True